In [1]:
from detectron2 import model_zoo
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer, ColorMode
import detectron2.data.transforms as T
from detectron2.data import DatasetMapper
import matplotlib.pyplot as plt
import numpy as np
import os
import json
import random
import cv2
import shutil


In [2]:
setup_logger()
dataset_dir = "dataset/"
train_dir = dataset_dir + "train"
val_dir = dataset_dir + "val"
test_dir = dataset_dir + "test"
classes = ["red", "pink", "yellow", "blue", "orange", "green"]
camera_rows = 516
camera_cols = 656

In [3]:
# allFileNames = np.arange(50)
# np.random.shuffle(allFileNames)
# train_ratio = 0.8
# is_new = False

# if not os.path.isdir(dataset_dir +'train/'):
#     os.makedirs(dataset_dir +'train/')
#     is_new = True
# if not os.path.isdir(dataset_dir +'val/'):
#     os.makedirs(dataset_dir +'val/')
#     is_new = True

# if is_new:
#     train_FileNames, val_FileNames = np.split(np.array(allFileNames), [int(len(allFileNames)* train_ratio)])
#     for name in train_FileNames:
#         shutil.copy(dataset_dir+"test_"+str(name)+".jpg", "dataset/train")
#         shutil.copy(dataset_dir+"test_"+str(name)+".json", "dataset/train")

#     for name in val_FileNames:
#         shutil.copy(dataset_dir+"test_"+str(name)+".jpg", "dataset/val")
#         shutil.copy(dataset_dir+"test_"+str(name)+".json", "dataset/val")


In [4]:
def get_data(img_dir):
    dataset_dicts = []
    for filename in [file for file in os.listdir(img_dir) if file.endswith('.json')]:
        json_file = os.path.join(img_dir, filename)
        with open(json_file) as f:
            img_label = json.load(f)

        record = {}
        
        img = os.path.join(img_dir, img_label["imagePath"])
        
        record["file_name"] = img
        record["height"] = camera_rows
        record["width"] = camera_cols
      
        shapes = img_label["shapes"]
        objs = []
        for shape in shapes:
            px = [a[0] for a in shape['points']]
            py = [a[1] for a in shape['points']]
            poly = [(x, y) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": classes.index(shape['label']),
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [5]:
for d in ["train", "val", "test"]:
    DatasetCatalog.register(d, lambda d=d: get_data(dataset_dir + d))
    MetadataCatalog.get(d).set(thing_classes=classes)

In [ ]:
dataset_dicts = get_data(train_dir)
train_metadata = MetadataCatalog.get("train")
for d in random.sample(dataset_dicts, 1):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=train_metadata)
    out = visualizer.draw_dataset_dict(d)
    plt.imshow(out.get_image()[..., ::-1][..., ::-1])

In [6]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.DEVICE = "cuda"
cfg.DATASETS.TRAIN = ("train",)
cfg.DATASETS.TEST = ("val")
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 3
cfg.INPUT.RANDOM_FLIP = "horizontal"
cfg.SOLVER.BASE_LR = 0.0025
cfg.SOLVER.MAX_ITER = 300
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.SOLVER.STEPS = []   
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes) 


In [7]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[12/02 19:50:51 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (7, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (7,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (24, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (24,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (6, 256, 1, 1) 

[12/02 19:50:53 d2.engine.train_loop]: Starting training from iteration 0


/home/samcbk/anaconda3/envs/ct213/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[12/02 19:50:54 d2.engine.hooks]: Total training time: 0:00:01 (0:00:00 on hooks)
[12/02 19:50:54 d2.utils.events]:  iter: 0    lr: N/A  max_mem: 2806M


KeyboardInterrupt: 

: 

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth") 

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

predictor = DefaultPredictor(cfg)

In [ ]:
i = 0
for d in [file for file in os.listdir(val_dir) if not file.endswith('.json')]:    
    im = cv2.imread(val_dir +"/"+ d)
    outputs = predictor(im) 
    v = Visualizer(im[:, :, ::-1])
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imwrite("val_" + str(i) + ".jpg", out.get_image()[..., ::-1])
    i += 1

In [ ]:
i = 0
for d in os.listdir(test_dir):    
    im = cv2.imread(test_dir + "/" + d)
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1])
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imwrite("test_" + str(i) + ".jpg", out.get_image()[..., ::-1])
    i += 1

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)